## Implement context-sensitive spelling correction

Here's a spelling correction model I found [here](https://norvig.com/spell-correct.html):

In [16]:
import re
from collections import Counter


def words(text): return re.findall(r"\w+", text.lower())


WORDS = Counter(words(open("data/big.txt").read()))


def P(word, n=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / n


def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)


def candidates(word):
    "Generate possible spelling corrections for word."
    return known([word]) or known(edits1(word)) or known(edits2(word)) or [word]


def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)


def edits1(word):
    "All edits that are one edit away from `word`."
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

The problem of this model is that it's not context-sensitive. To do that, I decided to add 2 BigGrams' dictionaries of words:

In [6]:
from collections import defaultdict


def create_bigram_dict(text, after: bool = True):
    """
    Creates dictionary of Bigram.
    
    Parameters:
        text: Input text to learn.
        after: If there should be dictionary for words after (True) or before (False)
    """
    bigram_dict = defaultdict(lambda: defaultdict(int))
    text_words = words(text)
    for i in range(len(text_words) - 1):
        if after:
            bigram_dict[text_words[i]][text_words[i + 1]] += 1
        else:
            bigram_dict[text_words[i + 1]][text_words[i]] += 1
    return bigram_dict


BIGRAM_DICT_AFTER = create_bigram_dict(open("data/big.txt").read())
BIGRAM_DICT_BEFORE = create_bigram_dict(open("data/big.txt").read(), False)

How do they work? Well, the first dictionary calculates number of appearance of each word AFTER another one.
Another dictionary makes vice versa, i.e. it stores number of appearance of each word BEFORE another one.
By doing that, we can calculate probability of appearance of some corrected word in the sentence.

In [13]:
def p_bi(word1: str = "", word2: str = "", word3: str = "", n=sum(WORDS.values())):
    """
    Probability of `word2` given `word1` before or after.
    
    Parameters:
        word1: The word before.
        word2: The actual word.
        word3: The word after.
        n: Number of words counted.
    """
    assert word2 != "", "The word to be corrected cannot be empty!"

    after = BIGRAM_DICT_AFTER[word1][word2] if word1 != "" else 0
    before = BIGRAM_DICT_BEFORE[word2][word3] if word3 != "" else 0
    return (after + before) / n


def correction_bi(prev_word: str = "", act_word: str = "", next_word: str = ""):
    """
    Most probable spelling correction for word given the previous word.
    
    Parameters:
        prev_word: The word before.
        act_word: The actual word.
        next_word: The word after.
    """
    return max(candidates(act_word), key=lambda word: p_bi(prev_word, word, next_word))


def correct_sentence_bi(sentence):
    """
    Corrects all words in the sentence.
    """
    text_words = words(sentence)
    corrected_words = [text_words[0]]

    for i in range(1, len(text_words)):
        corrected_words.append(correction_bi(corrected_words[i - 1], text_words[i]))

    return " ".join(corrected_words)

In [18]:
correct_sentence_bi("I hadd seen littl of Holmess lately")

'i had seen little of holmes lately'

Another problem can be is that the word before can be more in inluential than word after the corrected one or vice versa. Let's add nice coefficients for that:

In [19]:
def p_bi_coeffs(word1: str = "", word2: str = "", word3: str = "", n: int = sum(WORDS.values()), coeff_after: float = 1.0, coeff_before: float = 1.0):
    """
    Probability of `word2` given `word1` before or after.
    
    Parameters:
        word1: The word before.
        word2: The actual word.
        word3: The word after.
        n: Number of words counted.
        coeff_after: Coefficient of the probability to be after another one.
        coeff_before: Coefficient of the probability to be before another one.
    """
    assert word2 != "", "The word to be corrected cannot be empty!"

    after = BIGRAM_DICT_AFTER[word1][word2] if word1 != "" else 0
    before = BIGRAM_DICT_BEFORE[word2][word3] if word3 != "" else 0

    return (after * coeff_after + before * coeff_before) / n


def correction_bi_coeffs(prev_word: str = "", act_word: str = "", next_word: str = "", coeff_after: float = 1.0, coeff_before: float = 1.0):
    """
    Most probable spelling correction for word given the previous word.
    
    Parameters:
        prev_word: The first word.
        act_word: The second word.
        next_word: The third word.
        coeff_after: Coefficient of the probability to be after another one.
        coeff_before: Coefficient of the probability to be before another one.
    """
    return max(candidates(act_word), key=lambda word: p_bi_coeffs(prev_word, word, next_word, coeff_after=coeff_after, coeff_before=coeff_before))

In [24]:
from bayes_opt import BayesianOptimization
from random import choice


def test_acc(coeff1: float = 1.0, coeff2: float = 1.0):
    corrected_words = 0
    total_words = 0

    with open("data/big.txt") as f:
        for line in f:
            text_words = words(line)
            for i in range(1, len(text_words) - 1):
                uncorrect = text_words[i] + choice("qwertyuiopasdfghjklzxcvbnm")
                corrected_word = correction_bi_coeffs(text_words[i - 1], uncorrect, text_words[i + 1], coeff1, coeff2)
                if corrected_word == text_words[i]:
                    corrected_words += 1
                total_words += 1

    return corrected_words / total_words


optimizer = BayesianOptimization(
    f=test_acc,
    pbounds={"coeff1": (0, 1), "coeff2": (0, 1)},
    random_state=1
)

optimizer.maximize(init_points=5, n_iter=20)
optimizer.max["params"]["coeff1"], optimizer.max["params"]["coeff2"]

|   iter    |  target   |  coeff1   |  coeff2   |
-------------------------------------------------
| 1         | 0.8798    | 0.417     | 0.7203    |
| 2         | 0.8478    | 0.0001144 | 0.3023    |
| 3         | 0.8889    | 0.1468    | 0.09234   |
| 4         | 0.8795    | 0.1863    | 0.3456    |
| 5         | 0.8815    | 0.3968    | 0.5388    |
| 6         | 0.89      | 0.1575    | 0.08241   |
| 7         | 0.8913    | 0.3302    | 0.1536    |
| 8         | 0.8725    | 0.1493    | 0.6288    |
| 9         | 0.9065    | 0.489     | 0.0       |
| 10        | 0.907     | 0.6802    | 0.0       |
| 11        | 0.894     | 0.7451    | 0.2403    |
| 12        | 0.9061    | 0.9905    | 0.007976  |
| 13        | 0.8843    | 1.0       | 1.0       |
| 14        | 0.8948    | 1.0       | 0.2508    |
| 15        | 0.8758    | 0.1484    | 0.4067    |
| 16        | 0.9069    | 0.8285    | 0.0005093 |
| 17        | 0.9065    | 0.8269    | 0.00294   |
| 18        | 0.8893    | 1.0       | 0.5847    |


(0.6802245829897484, 0.0)

As you can see, the word after does no make ane influence on the corrected word.

## Evaluate on a test set
Here's an evaluation of the final model I created.
To test the model, I add random letter to the end of each word from the test text.
We count number of right corrections of that and measure accuracy.

In [18]:
test_acc(optimizer.max["params"]["coeff1"], optimizer.max["params"]["coeff2"])